This notebok is made wrt. Time series forecasting notebook from Tensorflow https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/time_series.ipynb

In [ ]:
import datetime
import os

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns

import IPython
import IPython.display

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam



In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# 1. Load the data


In [ ]:

path = "/home/guts/Documents/accel_data/2024-10-30_accel_data.csv"
merged_data = pd.read_csv(path)
merged_data.index = pd.to_datetime(merged_data['Timestamp'])
merged_data.drop('Timestamp', axis=1, inplace=True)
merged_data = merged_data[4::5] 
print(merged_data.head())


In [ ]:
# Plot the data
_ = merged_data.plot(subplots=True, figsize=(15, 10))

In [ ]:
merged_data.describe().transpose()

# 2. Feature Engineering

### Timestamp encoding (Sin/Cos tranformation)

In this step, we convert the timestamp into cyclic features using sine and cosine transformations. This is crucial for capturing the cyclical nature of time (e.g., minutes, hours, days) in the data, which helps the model correlate the raw sensor data with time, as the machines often operate in recurring cycles. This makes the LsTM aware of the cyclic nature of time.

In [ ]:

timestamp_s = merged_data.index.map(pd.Timestamp.timestamp)
# seconds in a :
minute = 60
hour = minute*60
day = hour*24
week = 7*day

merged_data['sin_minute'] = np.sin(2*np.pi*timestamp_s/minute)
merged_data['cos_minute'] = np.cos(2*np.pi*timestamp_s/minute)

merged_data['sin_hour'] = np.sin(2*np.pi*timestamp_s/hour)
merged_data['cos_hour'] = np.cos(2*np.pi*timestamp_s/hour)

merged_data['sin_day'] = np.sin(2*np.pi*timestamp_s/day)
merged_data['cos_day'] = np.cos(2*np.pi*timestamp_s/day)  

print(merged_data.head())

plt.figure(figsize=(15, 5))
plt.plot(np.array(merged_data['sin_minute'])[0:60])
plt.plot(np.array(merged_data['cos_minute'])[0:60])
plt.xlabel('Time [sec]')
plt.title('Time of minute signal')

**FFT(Fourier transform)**

A Fast Fourier transform (FFT) is applied to the feature to analyze the frequency component:

In [ ]:
# FFT
fft = tf.signal.rfft(merged_data['Linear x'])
f_per_dataset = np.arange(0, len(fft))

# Number of samples and total duration (in seconds)
n_samples_s = len(merged_data['Linear x'])  # Total number of samples
seconds_per_day = 24 * 60 * 60  # Seconds in a day
hours_per_day = 24  # Hours in a day

# Calculate the dataset's duration in days
days_per_dataset = n_samples_s / seconds_per_day

# Convert frequencies to cycles per day
f_per_day = f_per_dataset / days_per_dataset

# Plotting
plt.step(f_per_day, np.abs(fft))
plt.xscale('log')
plt.ylim(0, 500)  # Adjust for your amplitude range
plt.xlim([0.1, max(plt.xlim())])

# Add X-ticks for "1/day" and "1/hour"
plt.xticks([1, 24], labels=['1/hour', '1/day'])

# Labels
plt.xlabel('Frequency (log scale)')
plt.ylabel('Amplitude')
plt.title('Fourier Transform of Sensor Data')

# Show the plot
plt.show()


**X-axis (Frequency):**
The frequency is shown in logarithmic scale, ranging from lower frequencies (e.g., 1/hour) to higher frequencies (e.g., 1/day). This scale shows how often events or patterns repeat over time.

* 1/day: Represents patterns or trends that repeat approximately once per day.
* 1/hour: Represents events repeating about once per hour.

**Y-axis (Amplitude):**
The amplitude represents the strength or magnitude of each frequency component. Higher amplitudes indicate more significant variations at that particular frequency.

**Key Observations:**
The high amplitude at lower frequencies (around 1/day) indicates that the sensor data has strong periodic patterns at the daily level. This suggests that the sensor readings likely have a cyclical trend related to the daily operation of the machine.
As we move to higher frequencies (closer to 1/hour), the amplitude decreases, but there are still some smaller peaks. These could correspond to shorter, but less significant, cyclical patterns within each day or hour.



# 3. Normalization and Batching
The data is normalized and split into windows for the LSTM to process. A sliding window approach is used to feed sequential time segments to the model.

**Split the Data**

First the data is split into _test, train_ and _val_ datasets.

In [ ]:
column_indices = {name: i for i, name in enumerate(merged_data.columns)}

n = len(merged_data)
train_data = merged_data[0:int(n*0.7)]
val_data = merged_data[int(n*0.7):int(n*0.9)]
test_data = merged_data[int(n*0.9):]

num_features = merged_data.shape[1]
num_features


**Normalize the data**

In [ ]:
train_mean = train_data.mean()
train_std = train_data.std()


train_data = (train_data - train_mean) / train_std
val_data = (val_data - train_mean) / train_std
test_data = (test_data - train_mean) / train_std

merged_data_std = (merged_data - train_mean) / train_std
merged_data_std = merged_data_std.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))

ax = sns.violinplot(x='Column', y='Normalized', data=merged_data_std) 
_ = ax.set_xticklabels(merged_data.keys(), rotation=90)

**Data Windowing**

In [ ]:
# Idexes and offsets
class WindowGenerator:
    def __init__(self, input_width, label_width, shift, train_df=train_data,val_df=val_data, test_df=test_data, label_columns=None):
        
        # Store the raw data
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in 
                                          enumerate(label_columns)}

        self.column_indices = {name: i for i, name in 
                               enumerate(train_df.columns)}

        # Work out the window parameters
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
    

In [ ]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window


In [ ]:

def plot(self, model=None, plot_col='Linear x', max_subplots=3):
    inputs, labels = self.example
    plt.figure(figsize=(12, 8))
    plot_col_index = self.column_indices[plot_col]
    max_n = min(max_subplots, len(inputs))
    for n in range(max_n):
        plt.subplot(max_n, 1, n+1)
        plt.ylabel(f'{plot_col} [normed]')
        plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                 label='Inputs', marker='.', zorder=-10)
        
        if self.label_columns:
            label_col_index = self.label_columns_indices.get(plot_col, None)
        else:
            label_col_index = plot_col_index

        if label_col_index is None:
            continue

        plt.scatter(self.label_indices, labels[n, :, label_col_index],
                    edgecolors='k', label='Labels', c='#2ca02c', s=64)
        
        if model is not None:
            predictions = model(inputs)
            plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                        marker='X', edgecolors='k', label='Predictions',
                        c='#ff7f0e', s=64)
            
        if n == 0:
            plt.legend()

    plt.xlabel('Time [s]')

WindowGenerator.plot = plot

In [ ]:
# Create the dataset

def make_dataset(self, data):   
    data = np.array(data, dtype=np.float16)
    ds  = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=False,
        batch_size=32,)
    ds = ds.map(self.split_window)
    return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
@property
def train(self):
    return self.make_dataset(self.train_df)

@property
def val(self):
    return self.make_dataset(self.val_df)

@property
def test(self):
    return self.make_dataset(self.test_df)

@property
def example(self):
    result = getattr(self, '_example', None)
    if result is None:
        result = next(iter(self.train))
        self._example = result
    return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example


In [ ]:
wide_window = WindowGenerator(
    input_width=100, label_width=100, shift=1,
    label_columns=['Linear x'])

wide_window


The above cells returns the wide window created but only for the specified feature. 

In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

## Multi Output model

Here, we will create a wide window with multiple feature as outputs as well.

In [ ]:
%%time
wide_window = WindowGenerator(
    input_width=10, label_width=10, shift=1)

lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=num_features)
])

history = compile_and_fit(lstm_model, wide_window)

IPython.display.clear_output()
val_performance['LSTM'] = lstm_model.evaluate( wide_window.val, return_dict=True)
performance['LSTM'] = lstm_model.evaluate( wide_window.test, verbose=0, return_dict=True)

print()


In [ ]:
wide_window.plot(lstm_model, plot_col='Angular z')

## Residual connection

In [ ]:
class ResidualWrapper(tf.keras.Model):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def call(self, inputs, *args, **kwargs):
    delta = self.model(inputs, *args, **kwargs)

    # The prediction for each time step is the input
    # from the previous time step plus the delta
    # calculated by the model.
    return inputs + delta


In [ ]:
%%time
residual_lstm = ResidualWrapper(
    tf.keras.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dense(
        num_features,
        # The predicted deltas should start small.
        # Therefore, initialize the output layer with zeros.
        kernel_initializer=tf.initializers.zeros())
]))

history = compile_and_fit(residual_lstm, wide_window)

IPython.display.clear_output()
val_performance['Residual LSTM'] = residual_lstm.evaluate(wide_window.val, return_dict=True)
performance['Residual LSTM'] = residual_lstm.evaluate(wide_window.test, verbose=0, return_dict=True)
print()


In [ ]:
wide_window.plot(residual_lstm)

## Multistep models

In [ ]:
OUT_STEPS = 50
multi_window = WindowGenerator(input_width=50,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS)

multi_window.plot()
multi_window

### RNN

In [ ]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(64, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])      

history = compile_and_fit(multi_lstm_model, multi_window)

IPython.display.clear_output()

multi_val_performance = multi_lstm_model.evaluate(multi_window.val)
multi_performance = multi_lstm_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_lstm_model)

### Autoregressive RNN

In [ ]:
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(num_features)

In [ ]:
feedback_model = FeedBack(units=64, out_steps=OUT_STEPS)

In [ ]:
def warmup(self, inputs):
  # inputs.shape => (batch, time, features)
  # x.shape => (batch, lstm_units)
  x, *state = self.lstm_rnn(inputs)

  # predictions.shape => (batch, features)
  prediction = self.dense(x)
  return prediction, state

FeedBack.warmup = warmup

In [ ]:
prediction, state = feedback_model.warmup(multi_window.example[0])
prediction.shape

In [ ]:
def call(self, inputs, training=None):
  # Use a TensorArray to capture dynamically unrolled outputs.
  predictions = []
  # Initialize the lstm state
  prediction, state = self.warmup(inputs)

  # Insert the first prediction
  predictions.append(prediction)

  # Run the rest of the prediction steps
  for n in range(1, self.out_steps):
    # Use the last prediction as input.
    x = prediction
    # Execute one lstm step.
    x, state = self.lstm_cell(x, states=state,
                              training=training)
    # Convert the lstm output to a prediction.
    prediction = self.dense(x)
    # Add the prediction to the output
    predictions.append(prediction)

  # predictions.shape => (time, batch, features)
  predictions = tf.stack(predictions)
  # predictions.shape => (batch, time, features)
  predictions = tf.transpose(predictions, [1, 0, 2])
  return predictions

FeedBack.call = call

In [ ]:
print('Output shape (batch, time, features): ', feedback_model(multi_window.example[0]).shape)

In [ ]:
multi_val_performance = {}
multi_performance = {}

history = compile_and_fit(feedback_model, multi_window)

IPython.display.clear_output()

multi_val_performance['AR LSTM'] = feedback_model.evaluate(multi_window.val)
multi_performance['AR LSTM'] = feedback_model.evaluate(multi_window.test, verbose=0)

multi_window.plot(feedback_model)